In [1]:
# Dependencies
import pymongo
import datetime
import pandas as pd
from bson.json_util import dumps

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.se_db

# Declare the collections
players = db.players_db
nonplayers = db.nonplayers_db
games = db.games_db

### Pull data

In [2]:
player_path = "datacsvs/CleanPlayerData.csv"
playerdata = pd.read_csv(player_path)

nonplayer_path = "datacsvs/NonPlayerData.csv"
npdata = pd.read_csv(nonplayer_path)

gamepath = 'datacsvs/CleanGameData.csv'
game_data = pd.read_csv(gamepath)

### Set up player & game IDs

In [3]:
realdata = playerdata[playerdata['Alignment'] != '0'] # eventually change to playerdata
playerlist = realdata['person'].unique()
player_ids = {p:(i+1) for i,p in enumerate(playerlist)}

gamelist = realdata['Game'].unique()
game_ids = []
for i, g in enumerate(gamelist):
    if type(g) == str:
        curr_game = {
            'str':g,
            'id':i,
            'format':g[0:2],
            'num':g[2:]
        }
        game_ids.append(curr_game)

game_ids_df = pd.DataFrame(game_ids)

### Players (data test) into mongo

In [4]:
players.delete_many({})

stats_list = []
for p in realdata.iterrows():
    row = p[1]


    try: 
        curr_id = int(game_ids_df[game_ids_df['str'] == row['Game']].index[0])
    except:
        print(row)
        
    pgstat = {
        'player_id': player_ids.get(row['person'], 0),
        'player_name': row['person'],
        'game_id': curr_id,
        'game_str': row['Game'],
        'alignment': row['Alignment'],
        'fOutcome': row['fOutcome'],       
    }
    if type(row['Death']) != str:
        pgstat['death'] = '0'
    else:
        pgstat['death'] = row['Death']
    
    if row['HitNum'] != '-':
        pgstat['HitNum'] = row['HitNum']
        
        if row['Hit1'] != '-' :
            pgstat['Hit1'] = row['Hit1']
            pgstat['HitL'] = row['HitL']
            
    if row['Inactive'] == 'Y':
        pgstat['inactive'] = True
    else:
        pgstat['inactive'] = False
        
    if row['Role'] != '-':
        pgstat['role'] = row['Role']
    
    
    stats_list.append(pgstat)
    players.insert_one(pgstat)

In [5]:
json_player_data = dumps(list(players.find()))
 
with open('datajsons/playerdata.json', 'w') as file: 
    file.write(json_player_data) 

In [6]:
# # Verify results:
# results = players.find()
# for result in results:
#     print(result)

### Nonplayers (data test) into mongo

In [7]:
nonplayers.delete_many({})
# @TODO: rename row2 and currid2 to more descriptive vars
np_stats = []
for n in npdata.iterrows():
    row2 = n[1]

    try: 
        curr_id2 = int(game_ids_df[game_ids_df['str'] == row2['Game']].index[0])
    except:
        print(row)
        
    npgstat = {
        'player_id': player_ids.get(row2['person'], 0),
        'player_name': row2['person'],
        'game_id': curr_id2,
        'game_str': row2['Game'],
                
    }
            
    if row2['Spec'] == 'Y':
        npgstat['GM'] = False
        npgstat['spec'] = True
        npgstat['IM'] = False
    elif row2['GM'] == 'Y':
        npgstat['GM'] = True
        npgstat['spec'] = False
        npgstat['IM'] = False
    else:
        npgstat['GM'] = False
        npgstat['spec'] = False
        npgstat['IM'] = True

    
    np_stats.append(npgstat)
    nonplayers.insert_one(npgstat)
    

In [8]:
# results = nonplayers.find()
# for result in results:
#     print(result)

In [9]:
json_nonplayer_data = dumps(list(nonplayers.find()))
with open('datajsons/nonplayerdata.json', 'w') as file2: 
    file2.write(json_nonplayer_data) 

### Game data (game stats) into mongo

In [10]:
players_with_ids = realdata.merge(game_ids_df, left_on='Game', right_on='str')
grouped_pd = players_with_ids.groupby('id', sort = False)

In [11]:
# num players
broken_games = grouped_pd['Broken'].value_counts()
game_ids_df['num_players'] = list(broken_games)

In [12]:
# # Broken
# broken_games = grouped_pd['Broken'].value_counts().index
# broken_list = list(zip(*broken_games))[1]
# game_ids_df['Broken'] = [True if b == 'Y' else False for b in broken_list_yn]

In [13]:
# alignment, faction outcome, death, inactivity, broken? - from PlayerData
alignment_counts = grouped_pd['Alignment'].value_counts()
foutcome_counts = grouped_pd['fOutcome'].value_counts()
death_counts = grouped_pd['Death'].value_counts()
inactive_counts = grouped_pd['Inactive'].value_counts()
broken_counts = grouped_pd['Broken'].value_counts()

counts = [alignment_counts, foutcome_counts, death_counts, inactive_counts, broken_counts]
count_names = ['_align', '_win', '_death', '_inactive', '_broken']

for i, count in enumerate(counts): 
    print(count_names[i])
    count_df = count.unstack().fillna(0)
    game_ids_df = game_ids_df.merge(count_df, how='left', left_index=True, right_index=True, suffixes = ('', count_names[i]))



_align
_win
_death
_inactive
_broken


In [14]:
# Setting, complexity, # of cycles - from GameData file
relevant_gd = game_data[['Setting', 'Game Complexity', '# of Cycles']]
relevant_gd = relevant_gd.fillna(0)
relevant_gd = relevant_gd[relevant_gd['Setting'] != 0] # relies on apparent fact that setting is always filled in when any of them are

game_ids_df = game_ids_df.merge(relevant_gd, how='left', left_index=True, right_index=True)

In [17]:
# INTO MONGO

games.delete_many({})

game_stats = []
align_list = ['G', 'E', 'B', 'F', 'D', 'M', 'N', 'C', 'S']
outcome_list = ['L', 'W', 'D_win']
death_list = ['E_death', 'L_death', 'V', 'S_death', 'F_death', 'N_death', 'M_death', 'I', 'D_death', 'O']

for n in game_ids_df.iterrows():
    item = n[1]
 
    aligns = {k:item[k] for k in align_list if item[k] > 0}
    outcomes = {k:item[k] for k in outcome_list if item[k] > 0}
    deaths = {k:item[k] for k in death_list if item[k] > 0}
        
        
    gstat = {

        'game_id': item['id'],
        'game_str': item['str'],
        'format': item['format'],
        'game_num': item['num'],
#         'broken': item['Broken'],
        'num_players': item['num_players'],
        'alignment_counts': aligns,
        'outcome_counts': outcomes,
        'status_counts': deaths,
        'inactives': item['Y'],
        'broken': item['Y_broken']
            
        }
                
    game_stats.append(gstat)
    
    games.insert_one(gstat)
           

In [20]:
# results = games.find()
# for result in results:
#     print(result)

{'_id': ObjectId('5ffe83b3f720666e968130c0'), 'game_id': 0, 'game_str': 'LG1', 'format': 'LG', 'game_num': '1', 'num_players': 16, 'alignment_counts': {'G': 12.0, 'E': 4.0}, 'outcome_counts': {'L': 12.0, 'W': 4.0}, 'status_counts': {'E_death': 5.0, 'L_death': 4.0, 'V': 1.0, 'S_death': 6.0}, 'inactives': 0.0, 'broken': 0.0}
{'_id': ObjectId('5ffe83b3f720666e968130c1'), 'game_id': 1, 'game_str': 'LG2', 'format': 'LG', 'game_num': '2', 'num_players': 29, 'alignment_counts': {'G': 24.0, 'E': 1.0, 'B': 4.0}, 'outcome_counts': {'L': 24.0, 'W': 5.0}, 'status_counts': {'E_death': 14.0, 'L_death': 7.0, 'V': 2.0, 'S_death': 4.0, 'F_death': 2.0}, 'inactives': 0.0, 'broken': 0.0}
{'_id': ObjectId('5ffe83b3f720666e968130c2'), 'game_id': 2, 'game_str': 'LG3', 'format': 'LG', 'game_num': '3', 'num_players': 22, 'alignment_counts': {'G': 17.0, 'E': 5.0}, 'outcome_counts': {'L': 5.0, 'W': 17.0}, 'status_counts': {'E_death': 3.0, 'L_death': 7.0, 'V': 1.0, 'S_death': 8.0, 'F_death': 3.0}, 'inactives': 0.

In [ ]:
# @TODO
# 'Eliminators Lynched', 'Eliminators Vig'd' - breakdown by alignment + death type?

['mod', 'GM(s)', 'Spectator(s)'] # from nonplayer data
# start and end dates